In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

/tmp/ipykernel_22052/3175619481.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import csv

with open("./data/tokens_x86.csv") as f:
    reader = csv.reader(f)
    vocab = [row[1] for row in reader]

print(vocab)

['PHI', 'INLINEASM', 'INLINEASM_BR', 'CFI_INSTRUCTION', 'EH_LABEL', 'GC_LABEL', 'ANNOTATION_LABEL', 'KILL', 'EXTRACT_SUBREG', 'INSERT_SUBREG', 'IMPLICIT_DEF', 'SUBREG_TO_REG', 'COPY_TO_REGCLASS', 'DBG_VALUE', 'DBG_VALUE_LIST', 'DBG_INSTR_REF', 'DBG_PHI', 'DBG_LABEL', 'REG_SEQUENCE', 'COPY', 'BUNDLE', 'LIFETIME_START', 'LIFETIME_END', 'PSEUDO_PROBE', 'ARITH_FENCE', 'STACKMAP', 'FENTRY_CALL', 'PATCHPOINT', 'LOAD_STACK_GUARD', 'PREALLOCATED_SETUP', 'PREALLOCATED_ARG', 'STATEPOINT', 'LOCAL_ESCAPE', 'FAULTING_OP', 'PATCHABLE_OP', 'PATCHABLE_FUNCTION_ENTER', 'PATCHABLE_RET', 'PATCHABLE_FUNCTION_EXIT', 'PATCHABLE_TAIL_CALL', 'PATCHABLE_EVENT_CALL', 'PATCHABLE_TYPED_EVENT_CALL', 'ICALL_BRANCH_FUNNEL', 'MEMBARRIER', 'G_ASSERT_SEXT', 'G_ASSERT_ZEXT', 'G_ASSERT_ALIGN', 'G_ADD', 'G_SUB', 'G_MUL', 'G_SDIV', 'G_UDIV', 'G_SREM', 'G_UREM', 'G_SDIVREM', 'G_UDIVREM', 'G_AND', 'G_OR', 'G_XOR', 'G_IMPLICIT_DEF', 'G_PHI', 'G_FRAME_INDEX', 'G_GLOBAL_VALUE', 'G_CONSTANT_POOL', 'G_EXTRACT', 'G_UNMERGE_VALUES'

In [4]:
from llvm_ml.torch import BasicBlockDataset
banned_ids = []
dataset = BasicBlockDataset("./data/ryzen3600_v16.cbuf", vocab, masked=True, banned_ids=banned_ids, prefilter=True)

In [5]:
model = torch.load("./data/bert_ryzen3600.pt")

In [6]:
print(model)

MCBERT(
  (embedding): MCEmbedding(
    (embedding): Embedding(17859, 256)
    (pos_encoding): PositionalEncoding(256)
    (norm): LayerNorm(256, affine=True, mode=graph)
  )
  (encoders): ModuleList(
    (0-3): 4 x MCGraphEncoder(
      (norm1): LayerNorm(256, affine=True, mode=graph)
      (attention): MCGraphAttention(
        (key): Linear(in_features=256, out_features=256, bias=False)
        (query): Linear(in_features=256, out_features=256, bias=False)
        (value): Linear(in_features=256, out_features=256, bias=False)
        (proj): Linear(in_features=256, out_features=256, bias=False)
      )
      (norm2): LayerNorm(256, affine=True, mode=graph)
      (feed_forward): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=256, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (token_prediction): Linear(in_features=256, out_features=17859, b

In [7]:
model.eval()

MCBERT(
  (embedding): MCEmbedding(
    (embedding): Embedding(17859, 256)
    (pos_encoding): PositionalEncoding(256)
    (norm): LayerNorm(256, affine=True, mode=graph)
  )
  (encoders): ModuleList(
    (0-3): 4 x MCGraphEncoder(
      (norm1): LayerNorm(256, affine=True, mode=graph)
      (attention): MCGraphAttention(
        (key): Linear(in_features=256, out_features=256, bias=False)
        (query): Linear(in_features=256, out_features=256, bias=False)
        (value): Linear(in_features=256, out_features=256, bias=False)
        (proj): Linear(in_features=256, out_features=256, bias=False)
      )
      (norm2): LayerNorm(256, affine=True, mode=graph)
      (feed_forward): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=256, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (token_prediction): Linear(in_features=256, out_features=17859, b

In [8]:
from torch_geometric.loader import DataLoader
loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=6, drop_last=False)

model = model.to(device)
iterator = iter(loader)

In [117]:
bb, raw, mask_id, original_token = next(iterator)
bb = bb.to(device)

predicted = model(bb.x, bb.edge_index, bb.batch)
print(f"token_id = {original_token.item()}, value = {dataset.to_string(original_token.item())}")
print(mask_id.item())
print(predicted[0][mask_id.item()])
predicted_token = torch.argmax(predicted[0][mask_id.item()])
print(f"token_id = {predicted_token}, value = {dataset.to_string(predicted_token)}")

token_id = 1827, value = MOV64mr
12
tensor([ 4.9318, -2.7396, -3.3392,  ..., -0.5900,  0.7476, -3.3161],
       device='cuda:0', grad_fn=<SelectBackward0>)
token_id = 1827, value = MOV64mr
